In [1]:
import pandas as pd
import re
import numpy as np
import pyodbc 
import time
import datetime
import sqlite3
import logging
from sqlalchemy import create_engine

In [2]:
path = "..\Logs\coding_database_log.log"
logging.basicConfig(filename=path,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d,%(name)s,%(levelname)s,%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger("Log and Exception Notifications")
# logger.info("log test")

<>:1: SyntaxWarning: invalid escape sequence '\L'
<>:1: SyntaxWarning: invalid escape sequence '\L'
C:\Users\ylen\AppData\Local\Temp\ipykernel_24032\3497346663.py:1: SyntaxWarning: invalid escape sequence '\L'
  path = "..\Logs\coding_database_log.log"


## Read in log file 

In [3]:
import os

try:     # dynamically get all files in the Logs folder
    log_path = "..\Logs\\"
    log_files = os.listdir(log_path)

    # add the path to the begining of all file names
    log_files = [log_path + file for file in log_files]
    
    logger.info(f"Successfully got log paths")
except Exception as e:
    logger.error(f"Failed to get log paths.", exc_info=True) 
    print(e)

log_files

<>:4: SyntaxWarning: invalid escape sequence '\L'
<>:4: SyntaxWarning: invalid escape sequence '\L'
C:\Users\ylen\AppData\Local\Temp\ipykernel_24032\561205696.py:4: SyntaxWarning: invalid escape sequence '\L'
  log_path = "..\Logs\\"


['..\\Logs\\clinical_log.log',
 '..\\Logs\\coding_database_log.log',
 '..\\Logs\\foothold_log.log',
 '..\\Logs\\HR_log.log',
 '..\\Logs\\InSync_conn_log.log',
 '..\\Logs\\security_log.log',
 '..\\Logs\\viventium_demographics_log.log']

In [4]:
date_pattern = r'\d{4}-\d{2}-\d{2}'  # Example date pattern (yyyy-mm-dd)

all_exceptions_df = pd.DataFrame(columns=['Date','Program','Level','Message','Today'])
for path in log_files:
    log = path.split('\\')[-1]
    try:
        logger.info(f"Currently reading logs from {log}")
        # Read the log file into a DataFrame using regex pattern for splitting
        with open(path, 'r') as file:
            lines = file.read()
#             print("read file")

        entries = re.split(rf'(?={date_pattern})', lines)  # Split log entries based on date pattern

        # Create a DataFrame with the log entries
        df = pd.DataFrame({'Entry': entries})

        # split df into 4 columns based off of first four commas (Split columns based on info)
        df[["Date","Foo","Program","Level","Message"]] = df['Entry'].str.split(",",n=4,expand=True)
        # df

        # select date, program, level and message columns 
        df = df[["Date","Program","Level","Message"]]
        # select errors
        df = df[df["Level"] == "ERROR"]

        # Today's exceptions
        # get today's date
        today = pd.Timestamp(datetime.date.today())
        today = str(today.date())

        # check if Date of Exception contains today's date
        df['Today'] = df['Date'].apply(lambda entry: today in entry).astype('boolean')
        
        all_exceptions_df = pd.concat([all_exceptions_df,df])
        logger.info(f"Successfully read logs from {log}, concated with all exceptions")
    except Exception as e:
        logger.error(f"Failed to read logs from {log}", exc_info=True)
        print(e)

all_exceptions_df

C:\Users\ylen\AppData\Local\Temp\ipykernel_24032\2766351826.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exceptions_df = pd.concat([all_exceptions_df,df])


,Date,Program,Level,Message,Today
2814,2023-07-06 16:26:56,root,ERROR,Failed to push: emr_PatientDetails \n Tra...,False
2934,2023-07-07 07:18:21,root,ERROR,Failed to push: emr_PatientDetails \n Tra...,False
3054,2023-07-08 07:16:08,root,ERROR,Failed to push: emr_PatientDetails \n Tra...,False
3174,2023-07-09 07:16:16,root,ERROR,Failed to push: emr_PatientDetails \n Tra...,False
3294,2023-07-10 07:16:18,root,ERROR,Failed to push: emr_PatientDetails \n Tra...,False
...,...,...,...,...,...
1103,2024-12-30 15:00:23,root,ERROR,Error encountered in emr_PatientDetails: 'str'...,False
1157,2024-12-30 15:02:08,root,ERROR,Error encountered in emr_PatientDetails: 'bool...,False
1167,2024-12-30 15:04:42,root,ERROR,Error encountered in emr_PatientDetails: 'bool...,False
1232,2024-12-30 15:16:21,root,ERROR,Error encountered in emr_PatientDetails: Inval...,False


## Write to db 

In [5]:
# connect to db
conn = create_engine(r'mssql+pyodbc://@PYTHONSERVER\SQLEXPRESS/Coding_Logs?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes', fast_executemany=True)

table_name = "coding_errors"

# create table
# try:
#     c = conn.cursor()
#     c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (Program, TEXT,
#                                                            Level TEXT,
#                                                            Message TEXT)''')
#     conn.commit()
#     logger.info(f"Successfully created {table_name}.")
#     print("table created")
# except Exception as e:
#     logger.error(f"Failed to create {table_name}.", exc_info=True) 
#     print(e)

# push data into table
try:
    all_exceptions_df.to_sql(table_name, conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed data to {table_name}")
    print("data pushed")
except Exception as e:
    logger.error(f"Failed to push data to {table_name}.", exc_info=True) 
    print(e)
    

data pushed


In [6]:
# conn.close()
# logging.shutdown()